### Comparing with other models like : Weighted logistic regression ,Random forest , Ada boost , XG boost , stacking model , Artificial neural netwrok

In [1]:
import pandas as pd 
import numpy as np

In [2]:
loan_csv_path = "/kaggle/input/pan-card-realfake-image/loan_applications.csv"
txn_csv_path = "/kaggle/input/pan-card-realfake-image/transactions.csv"

In [3]:
loan_df = pd.read_csv(loan_csv_path)
txn_df = pd.read_csv(txn_csv_path)

In [4]:
loan_df.head()

,application_id,customer_id,application_date,loan_type,loan_amount_requested,loan_tenure_months,interest_rate_offered,purpose_of_loan,employment_status,monthly_income,...,existing_emis_monthly,debt_to_income_ratio,property_ownership_status,residential_address,applicant_age,gender,number_of_dependents,loan_status,fraud_flag,fraud_type
0,c8bf0bea-70e6-4870-9125-41b8210c527f,CUST109427,2023-04-09,Business Loan,604000.0,12,11.66,Medical Emergency,Retired,34700.0,...,1100.0,3.17,Rented,"94/31, Sehgal Zila, Vadodara-380521, Anantapur...",28,Female,3,Approved,0,NaN
1,91224cec-3544-4bc7-ac15-a9792da54c02,CUST106146,2023-09-23,Car Loan,100000.0,240,13.62,Education,Unemployed,51600.0,...,0.0,0.00,Owned,"H.No. 00, Sheth Chowk, Ichalkaranji 006728, Im...",44,Other,3,Approved,0,NaN
2,4efcd02d-4a03-4ab7-9bd1-0ff430493d0c,CUST100674,2023-05-22,Education Loan,431000.0,60,11.40,Medical Emergency,Self-Employed,14800.0,...,4600.0,31.08,Rented,"H.No. 81, Dutta Path, Kozhikode-340301, Tadepa...",56,Other,4,Approved,0,NaN
3,a61337d4-ba04-4a68-b492-2cb8266e6ed7,CUST106466,2024-07-09,Car Loan,324000.0,120,10.36,Debt Consolidation,Self-Employed,28800.0,...,4000.0,13.89,Rented,"H.No. 022, Rege Road, Tiruvottiyur-927857, Aur...",27,Other,4,Declined,0,NaN
4,a8d1639e-170b-41b2-826a-55c7dae38d16,CUST112319,2023-11-20,Personal Loan,100000.0,36,14.14,Business Expansion,Salaried,43900.0,...,1100.0,2.51,Rented,"85/24, Bali Zila, Sambalpur 922071, Tumkur, Ke...",50,Other,0,Declined,0,NaN


In [5]:
txn_df.head()

,transaction_id,customer_id,transaction_date,transaction_type,transaction_amount,merchant_category,merchant_name,transaction_location,account_balance_after_transaction,is_international_transaction,device_used,ip_address,transaction_status,transaction_source_destination,transaction_notes,fraud_flag
0,2d7ddfd4-1112-4354-a2dd-fad94b45a850,CUST102188,2022-01-04 11:27:00,Bill Payment,2100.0,Dining,"Rattan, Khanna and Magar","Srikakulam, Himachal Pradesh",8211.0,0,Web,31.102.21.141,Success,BCHG80012468703731,Payment for Dining,0
1,ba998721-a612-4e0a-8354-46d001e9c39c,CUST107144,2022-01-03 23:52:00,UPI,1400.0,Travel,Chauhan LLC,"Giridih, Manipur",12858.0,0,Web,176.48.222.166,Success,CJPZ11841095072972,Payment for Travel,0
2,0b4ad8e2-40ac-49c9-b59a-c9efaaccc7a6,CUST102126,2022-01-15 06:54:00,Debit Card,1000.0,Entertainment,Halder-Sarma,"Avadi, Kerala",19268.0,0,ATM,181.62.105.161,Success,TLCG50761416960179,Payment for Entertainment,0
3,c4fce988-34c5-48db-8686-bfad0b3bc0b5,CUST109887,2022-01-08 07:57:00,Credit Card,10500.0,Utilities,"Anand, Khare and Balakrishnan","Navi Mumbai, Maharashtra",29585.0,0,ATM,121.157.246.80,Success,PRZJ55096553432036,Payment for Utilities,0
4,f9b8ca30-00e5-4322-8cff-eac0554cf2da,CUST108620,2022-01-02 10:43:00,Loan Disbursement,8100.0,Electronics,Zacharia and Sons,"Visakhapatnam, Gujarat",0.0,0,ATM,129.96.5.57,Success,NBAI29021393288369,Payment for Electronics,0


In [6]:
txn_agg = txn_df.groupby('customer_id').agg({
    'transaction_amount': ['sum','mean','max','min','std'],
    'transaction_id':'count',
    'is_international_transaction':'sum',
    'account_balance_after_transaction':['mean','min']
})
txn_agg.columns = ['_'.join(col).strip() for col in txn_agg.columns]
txn_agg.reset_index(inplace=True)

df = loan_df.merge(txn_agg, on='customer_id', how='left')

In [7]:
df.head()

,application_id,customer_id,application_date,loan_type,loan_amount_requested,loan_tenure_months,interest_rate_offered,purpose_of_loan,employment_status,monthly_income,...,fraud_type,transaction_amount_sum,transaction_amount_mean,transaction_amount_max,transaction_amount_min,transaction_amount_std,transaction_id_count,is_international_transaction_sum,account_balance_after_transaction_mean,account_balance_after_transaction_min
0,c8bf0bea-70e6-4870-9125-41b8210c527f,CUST109427,2023-04-09,Business Loan,604000.0,12,11.66,Medical Emergency,Retired,34700.0,...,NaN,13700.0,6850.000000,8800.0,4900.0,2757.716447,2.0,0.0,8332.500000,0.0
1,91224cec-3544-4bc7-ac15-a9792da54c02,CUST106146,2023-09-23,Car Loan,100000.0,240,13.62,Education,Unemployed,51600.0,...,NaN,18700.0,4675.000000,9800.0,900.0,3730.393187,4.0,0.0,31553.750000,13656.0
2,4efcd02d-4a03-4ab7-9bd1-0ff430493d0c,CUST100674,2023-05-22,Education Loan,431000.0,60,11.40,Medical Emergency,Self-Employed,14800.0,...,NaN,13100.0,3275.000000,7600.0,300.0,3607.746296,4.0,0.0,31112.250000,18545.0
3,a61337d4-ba04-4a68-b492-2cb8266e6ed7,CUST106466,2024-07-09,Car Loan,324000.0,120,10.36,Debt Consolidation,Self-Employed,28800.0,...,NaN,2900.0,1450.000000,2200.0,700.0,1060.660172,2.0,0.0,11021.500000,1476.0
4,a8d1639e-170b-41b2-826a-55c7dae38d16,CUST112319,2023-11-20,Personal Loan,100000.0,36,14.14,Business Expansion,Salaried,43900.0,...,NaN,35600.0,11866.666667,18500.0,3300.0,7782.244749,3.0,0.0,24092.333333,0.0


In [8]:
df=df.drop('transaction_amount_std',axis=1)

In [9]:
df = df.drop(columns=['application_id', 'customer_id', 'application_date']) 

In [11]:
 from sklearn.preprocessing import StandardScaler, LabelEncoder

In [12]:
# Fill missing numeric values with median
numeric_cols = df.select_dtypes(include=np.number).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Fill missing categorical values with mode
categorical_cols = df.select_dtypes(include='object').columns
df[categorical_cols] = df[categorical_cols].fillna(df[categorical_cols].mode().iloc[0])

# Encode categorical variables
le_dict = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    le_dict[col] = le

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 26 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   loan_type                               50000 non-null  int64  
 1   loan_amount_requested                   50000 non-null  float64
 2   loan_tenure_months                      50000 non-null  int64  
 3   interest_rate_offered                   50000 non-null  float64
 4   purpose_of_loan                         50000 non-null  int64  
 5   employment_status                       50000 non-null  int64  
 6   monthly_income                          50000 non-null  float64
 7   cibil_score                             50000 non-null  int64  
 8   existing_emis_monthly                   50000 non-null  float64
 9   debt_to_income_ratio                    50000 non-null  float64
 10  property_ownership_status               50000 non-null  in

In [14]:
leaky_cols = ['fraud_type', 'account_balance_after_transaction_mean', 'account_balance_after_transaction_min']
df = df.drop(columns=leaky_cols)

In [15]:
df = df.drop(columns=['transaction_id_count', 'residential_address'])

In [16]:
missing_values = df.isnull().sum()
print(missing_values)

loan_type                           0
loan_amount_requested               0
loan_tenure_months                  0
interest_rate_offered               0
purpose_of_loan                     0
employment_status                   0
monthly_income                      0
cibil_score                         0
existing_emis_monthly               0
debt_to_income_ratio                0
property_ownership_status           0
applicant_age                       0
gender                              0
number_of_dependents                0
loan_status                         0
fraud_flag                          0
transaction_amount_sum              0
transaction_amount_mean             0
transaction_amount_max              0
transaction_amount_min              0
is_international_transaction_sum    0
dtype: int64


In [17]:
df.head()

,loan_type,loan_amount_requested,loan_tenure_months,interest_rate_offered,purpose_of_loan,employment_status,monthly_income,cibil_score,existing_emis_monthly,debt_to_income_ratio,...,applicant_age,gender,number_of_dependents,loan_status,fraud_flag,transaction_amount_sum,transaction_amount_mean,transaction_amount_max,transaction_amount_min,is_international_transaction_sum
0,0,604000.0,12,11.66,4,1,34700.0,714,1100.0,3.17,...,28,0,3,0,0,13700.0,6850.000000,8800.0,4900.0,0.0
1,1,100000.0,240,13.62,2,5,51600.0,667,0.0,0.00,...,44,2,3,0,0,18700.0,4675.000000,9800.0,900.0,0.0
2,2,431000.0,60,11.40,4,3,14800.0,808,4600.0,31.08,...,56,2,4,0,0,13100.0,3275.000000,7600.0,300.0,0.0
3,1,324000.0,120,10.36,1,3,28800.0,647,4000.0,13.89,...,27,2,4,1,0,2900.0,1450.000000,2200.0,700.0,0.0
4,4,100000.0,36,14.14,0,2,43900.0,624,1100.0,2.51,...,50,2,0,1,0,35600.0,11866.666667,18500.0,3300.0,0.0


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

#  Define target and features
y = df['fraud_flag']
X = df.drop(columns=['fraud_flag'])

#  One-hot encode categorical variables 
cat_cols = X.select_dtypes(include=['object']).columns
X = pd.get_dummies(X, columns=cat_cols, drop_first=True)

#  Remove columns with constant values
X = X.loc[:, X.apply(pd.Series.nunique) > 1]

#  Compute VIF
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

#  Sort by highest VIF
vif_data = vif_data.sort_values(by='VIF', ascending=False).reset_index(drop=True)

#  Display VIF table
vif_data

,Feature,VIF
0,transaction_amount_mean,59.064629
1,cibil_score,55.936733
2,transaction_amount_max,33.890515
3,interest_rate_offered,28.154801
4,applicant_age,11.653487
5,transaction_amount_min,11.105964
6,transaction_amount_sum,10.700907
7,monthly_income,9.940214
8,existing_emis_monthly,6.489867
9,debt_to_income_ratio,4.701310


In [19]:
# Drop selected high-VIF columns
cols_to_drop = [
    'transaction_amount_sum',
    'transaction_amount_max',
    'transaction_amount_min',
    'cibil_score'
]

df = df.drop(columns=cols_to_drop)
df.columns


Index(['loan_type', 'loan_amount_requested', 'loan_tenure_months',
       'interest_rate_offered', 'purpose_of_loan', 'employment_status',
       'monthly_income', 'existing_emis_monthly', 'debt_to_income_ratio',
       'property_ownership_status', 'applicant_age', 'gender',
       'number_of_dependents', 'loan_status', 'fraud_flag',
       'transaction_amount_mean', 'is_international_transaction_sum'],
      dtype='object')

In [20]:
y = df['fraud_flag']
X = df.drop(columns=['fraud_flag'])

In [ ]:
#  Compute VIF
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

#  Sort by highest VIF
vif_data = vif_data.sort_values(by='VIF', ascending=False).reset_index(drop=True)
vif_data

,Feature,VIF
0,interest_rate_offered,17.775322
1,applicant_age,10.161412
2,monthly_income,8.948220
3,existing_emis_monthly,6.487737
4,debt_to_income_ratio,4.476918
5,loan_amount_requested,4.260820
6,purpose_of_loan,3.144281
7,employment_status,3.035580
8,transaction_amount_mean,2.983260
9,number_of_dependents,2.954617


In [22]:
cols_to_drop = ['interest_rate_offered']
df = df.drop(columns=cols_to_drop)

# Recalculate VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = df.drop(columns=['fraud_flag'])
X = pd.get_dummies(X, drop_first=True)
X = X.loc[:, X.apply(pd.Series.nunique) > 1]

vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_data = vif_data.sort_values(by='VIF', ascending=False).reset_index(drop=True)

vif_data

,Feature,VIF
0,applicant_age,8.834052
1,monthly_income,8.081200
2,existing_emis_monthly,6.487057
3,debt_to_income_ratio,4.287743
4,loan_amount_requested,4.109365
5,purpose_of_loan,3.066765
6,employment_status,2.978447
7,transaction_amount_mean,2.911104
8,number_of_dependents,2.883893
9,loan_type,2.861648


In [28]:
print(df['fraud_flag'].value_counts())

fraud_flag
0    48974
1     1026
Name: count, dtype: int64


In [23]:
y = df['fraud_flag']
X = df.drop(columns=['fraud_flag'])

### Weighted logistic regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# One-hot encode categorical columns 
X = pd.get_dummies(X, drop_first=True)

#  Train-test split 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
#  Weighted Logistic Regression to handle imbalance
log_reg = LogisticRegression(
    class_weight='balanced',   
    solver='liblinear',        
    max_iter=1000
)

# Fit the model
log_reg.fit(X_train, y_train)

#  Predictions
y_pred = log_reg.predict(X_test)

#  Evaluation
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[9794    1]
 [   0  205]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9795
           1       1.00      1.00      1.00       205

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000



In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Predictions
y_train_pred = log_reg.predict(X_train)
y_test_pred = log_reg.predict(X_test)

# Confusion Matrices
print("Confusion Matrix - Train Set:")
print(confusion_matrix(y_train, y_train_pred))
print("\nConfusion Matrix - Test Set:")
print(confusion_matrix(y_test, y_test_pred))

# Accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"\nTrain Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Full Classification Reports
print("\nClassification Report - Train Set:")
print(classification_report(y_train, y_train_pred))

print("Classification Report - Test Set:")
print(classification_report(y_test, y_test_pred))


Confusion Matrix - Train Set:
[[39178     1]
 [    0   821]]

Confusion Matrix - Test Set:
[[9794    1]
 [   0  205]]

Train Accuracy: 1.0000
Test Accuracy: 0.9999

Classification Report - Train Set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     39179
           1       1.00      1.00      1.00       821

    accuracy                           1.00     40000
   macro avg       1.00      1.00      1.00     40000
weighted avg       1.00      1.00      1.00     40000

Classification Report - Test Set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9795
           1       1.00      1.00      1.00       205

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000



### Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

#  Initialize Random Forest with balanced class weight
rf_clf = RandomForestClassifier(
    n_estimators=200,          # number of trees
    max_depth=None,             # let trees grow fully
    random_state=42,
    class_weight='balanced',    # handle imbalance
    n_jobs=-1                   # use all CPU cores
)

#  Fit model
rf_clf.fit(X_train, y_train)

#  Predictions
y_train_pred_rf = rf_clf.predict(X_train)
y_test_pred_rf = rf_clf.predict(X_test)

#  Confusion Matrices
print("Confusion Matrix - Train Set:")
print(confusion_matrix(y_train, y_train_pred_rf))
print("\nConfusion Matrix - Test Set:")
print(confusion_matrix(y_test, y_test_pred_rf))

#  Accuracy
train_accuracy_rf = accuracy_score(y_train, y_train_pred_rf)
test_accuracy_rf = accuracy_score(y_test, y_test_pred_rf)
print(f"\nTrain Accuracy: {train_accuracy_rf:.4f}")
print(f"Test Accuracy: {test_accuracy_rf:.4f}")

#  Classification Reports
print("\nClassification Report - Train Set:")
print(classification_report(y_train, y_train_pred_rf))
print("\nClassification Report - Test Set:")
print(classification_report(y_test, y_test_pred_rf))


Confusion Matrix - Train Set:
[[39179     0]
 [    0   821]]

Confusion Matrix - Test Set:
[[9795    0]
 [   0  205]]

Train Accuracy: 1.0000
Test Accuracy: 1.0000

Classification Report - Train Set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     39179
           1       1.00      1.00      1.00       821

    accuracy                           1.00     40000
   macro avg       1.00      1.00      1.00     40000
weighted avg       1.00      1.00      1.00     40000


Classification Report - Test Set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9795
           1       1.00      1.00      1.00       205

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000



### Ada boost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

#  Base estimator for AdaBoost (decision stump)
base_estimator = DecisionTreeClassifier(
    max_depth=1,            # weak learner
    class_weight='balanced', # handle imbalance in base learner
    random_state=42
)

#  Initialize AdaBoost
ada_clf = AdaBoostClassifier(
    base_estimator=base_estimator,
    n_estimators=200,       # number of weak learners
    learning_rate=0.5,
    random_state=42
)

#  Fit AdaBoost on training data
ada_clf.fit(X_train, y_train)

#  Predictions
y_train_pred_ada = ada_clf.predict(X_train)
y_test_pred_ada = ada_clf.predict(X_test)

#  Confusion Matrices
print("Confusion Matrix - Train Set:")
print(confusion_matrix(y_train, y_train_pred_ada))
print("\nConfusion Matrix - Test Set:")
print(confusion_matrix(y_test, y_test_pred_ada))

#  Accuracy
train_accuracy_ada = accuracy_score(y_train, y_train_pred_ada)
test_accuracy_ada = accuracy_score(y_test, y_test_pred_ada)
print(f"\nTrain Accuracy: {train_accuracy_ada:.4f}")
print(f"Test Accuracy: {test_accuracy_ada:.4f}")

# Classification Reports
print("\nClassification Report - Train Set:")
print(classification_report(y_train, y_train_pred_ada))
print("\nClassification Report - Test Set:")
print(classification_report(y_test, y_test_pred_ada))


Confusion Matrix - Train Set:
[[39179     0]
 [    0   821]]

Confusion Matrix - Test Set:
[[9795    0]
 [   0  205]]

Train Accuracy: 1.0000
Test Accuracy: 1.0000

Classification Report - Train Set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     39179
           1       1.00      1.00      1.00       821

    accuracy                           1.00     40000
   macro avg       1.00      1.00      1.00     40000
weighted avg       1.00      1.00      1.00     40000


Classification Report - Test Set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9795
           1       1.00      1.00      1.00       205

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000



/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


### XG boost

In [ ]:
import xgboost as xgb
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

#  Initialize XGBoost classifier
xgb_clf = xgb.XGBClassifier(
    n_estimators=200,         # number of trees
    max_depth=5,               # max depth of each tree
    learning_rate=0.1,
    scale_pos_weight=(y_train==0).sum() / (y_train==1).sum(),  # handle class imbalance
    objective='binary:logistic',
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

#  Fit XGBoost on training data
xgb_clf.fit(X_train, y_train)

#  Predictions
y_train_pred_xgb = xgb_clf.predict(X_train)
y_test_pred_xgb = xgb_clf.predict(X_test)

#  Confusion Matrices
print("Confusion Matrix - Train Set:")
print(confusion_matrix(y_train, y_train_pred_xgb))
print("\nConfusion Matrix - Test Set:")
print(confusion_matrix(y_test, y_test_pred_xgb))

# Accuracy
train_accuracy_xgb = accuracy_score(y_train, y_train_pred_xgb)
test_accuracy_xgb = accuracy_score(y_test, y_test_pred_xgb)
print(f"\nTrain Accuracy: {train_accuracy_xgb:.4f}")
print(f"Test Accuracy: {test_accuracy_xgb:.4f}")

#  Classification Reports
print("\nClassification Report - Train Set:")
print(classification_report(y_train, y_train_pred_xgb))
print("\nClassification Report - Test Set:")
print(classification_report(y_test, y_test_pred_xgb))


Confusion Matrix - Train Set:
[[39179     0]
 [    0   821]]

Confusion Matrix - Test Set:
[[9795    0]
 [   0  205]]

Train Accuracy: 1.0000
Test Accuracy: 1.0000

Classification Report - Train Set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     39179
           1       1.00      1.00      1.00       821

    accuracy                           1.00     40000
   macro avg       1.00      1.00      1.00     40000
weighted avg       1.00      1.00      1.00     40000


Classification Report - Test Set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9795
           1       1.00      1.00      1.00       205

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000



### Stacking model

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

#  Define base models 
base_estimators = [
    ('lr', LogisticRegression(class_weight='balanced', solver='liblinear', max_iter=1000)),
    ('rf', RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42)),
    ('ada', AdaBoostClassifier(
        base_estimator=DecisionTreeClassifier(max_depth=1, class_weight='balanced', random_state=42),
        n_estimators=100, learning_rate=0.5, random_state=42
    )),
    ('xgb', xgb.XGBClassifier(
        n_estimators=200,
        max_depth=5,
        learning_rate=0.1,
        scale_pos_weight=(y_train==0).sum() / (y_train==1).sum(),
        objective='binary:logistic',
        random_state=42,
        use_label_encoder=False,
        eval_metric='logloss'
    ))
]

#  Define stacking classifier with Logistic Regression as meta-model
stack_clf = StackingClassifier(
    estimators=base_estimators,
    final_estimator=LogisticRegression(),
    cv=5,                   # 5-fold cross-validation for meta-model training
    n_jobs=-1,              # parallel computation
    passthrough=True        # pass base model predictions to meta-model
)

#  Fit stacking model
stack_clf.fit(X_train, y_train)

#  Predictions
y_train_pred_stack = stack_clf.predict(X_train)
y_test_pred_stack = stack_clf.predict(X_test)

#  Confusion Matrices
print("Confusion Matrix - Train Set:")
print(confusion_matrix(y_train, y_train_pred_stack))
print("\nConfusion Matrix - Test Set:")
print(confusion_matrix(y_test, y_test_pred_stack))

#  Accuracy
train_accuracy_stack = accuracy_score(y_train, y_train_pred_stack)
test_accuracy_stack = accuracy_score(y_test, y_test_pred_stack)
print(f"\nTrain Accuracy: {train_accuracy_stack:.4f}")
print(f"Test Accuracy: {test_accuracy_stack:.4f}")

#  Classification Reports
print("\nClassification Report - Train Set:")
print(classification_report(y_train, y_train_pred_stack))
print("\nClassification Report - Test Set:")
print(classification_report(y_test, y_test_pred_stack))

/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/en

Confusion Matrix - Train Set:
[[39179     0]
 [  821     0]]

Confusion Matrix - Test Set:
[[9795    0]
 [ 205    0]]

Train Accuracy: 0.9795
Test Accuracy: 0.9795

Classification Report - Train Set:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     39179
           1       0.00      0.00      0.00       821

    accuracy                           0.98     40000
   macro avg       0.49      0.50      0.49     40000
weighted avg       0.96      0.98      0.97     40000


Classification Report - Test Set:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      9795
           1       0.00      0.00      0.00       205

    accuracy                           0.98     10000
   macro avg       0.49      0.50      0.49     10000
weighted avg       0.96      0.98      0.97     10000



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classif

### Artificial neural network

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

#  Convert data to tensors
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # scale features
y_array = y.values.reshape(-1, 1)   # reshape to column

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values.reshape(-1,1), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values.reshape(-1,1), dtype=torch.float32)

#  Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

#  Define the ANN
class FraudANN(nn.Module):
    def __init__(self, input_dim):
        super(FraudANN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.model(x)

#  Initialize model
input_dim = X_train.shape[1]
model = FraudANN(input_dim)

#  Loss function with class weights
# Compute class weights
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = torch.tensor([weights[1], weights[0]])  # 1 = fraud, 0 = non-fraud

criterion = nn.BCELoss()  # for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

#  Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader):.4f}")

#  Evaluate on test set
model.eval()
with torch.no_grad():
    y_train_pred = (model(X_train_tensor) >= 0.5).float()
    y_test_pred = (model(X_test_tensor) >= 0.5).float()

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print("Train Confusion Matrix:")
print(confusion_matrix(y_train, y_train_pred))
print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))

print("\nTrain Accuracy:", accuracy_score(y_train, y_train_pred))
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))

print("\nTrain Classification Report:")
print(classification_report(y_train, y_train_pred, zero_division=0))
print("\nTest Classification Report:")
print(classification_report(y_test, y_test_pred, zero_division=0))

Epoch 1/20, Loss: 2.5510
Epoch 2/20, Loss: 2.0525
Epoch 3/20, Loss: 2.0525
Epoch 4/20, Loss: 2.0525
Epoch 5/20, Loss: 2.0525
Epoch 6/20, Loss: 2.0525
Epoch 7/20, Loss: 2.0525
Epoch 8/20, Loss: 2.0525
Epoch 9/20, Loss: 2.0525
Epoch 10/20, Loss: 2.0525
Epoch 11/20, Loss: 2.0525
Epoch 12/20, Loss: 2.0525
Epoch 13/20, Loss: 2.0525
Epoch 14/20, Loss: 2.0525
Epoch 15/20, Loss: 2.0525
Epoch 16/20, Loss: 2.0525
Epoch 17/20, Loss: 2.0525
Epoch 18/20, Loss: 2.0525
Epoch 19/20, Loss: 2.0525
Epoch 20/20, Loss: 2.0525
Train Confusion Matrix:
[[39179     0]
 [  821     0]]

Test Confusion Matrix:
[[9795    0]
 [ 205    0]]

Train Accuracy: 0.979475
Test Accuracy: 0.9795

Train Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     39179
           1       0.00      0.00      0.00       821

    accuracy                           0.98     40000
   macro avg       0.49      0.50      0.49     40000
weighted avg       0.96      0.98